# Unsolvable Loop Analysis

In [2]:
!cat loops/markov-triples-random.prob

a, b, c = 1, 1, 2
while true:
    d = Bernoulli(1/2)
    if d == 1:
        a, b, c = a, 3*a*b - c, b
    else:
        a, b, c = b, 3*b*c - a, c
    end
end


In [5]:
!python ../polar.py loops/markov-triples-random.prob --goals "E(c)" --solvability_check


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Traceback (most recent call last):
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 35, in <module>
    main()
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 27, in main
    raise e
  File "/home/marcel/Repos/polar/documentation/../polar.py", line 25, in main
    action(benchmark)
  File "/home/marcel/Repos/polar/cli/actions/goals_action.py", line 54, in __call__
    self.handle_all_goals()
  File "/home/marcel/Repos/polar/cli/act

In [10]:
!python ../polar.py loops/markov-triples-random.prob --synth_unsolv_inv --inv_deg 2


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Searching for invariants for special case k = 1..
No invariant found with degree 2 and k=1

Searching for invariants, general case..
E(-_u27*(a - c)*(a + c)) = 3*2**(-n)*_u27
E(-(a - c)*(_u24 + _u27*a + _u27*c)) = 2**(-n)*(_u24 + 3*_u27)
Elapsed time: 0.9417839050292969 s


In [11]:
!python ../polar.py loops/markov-triples-random.prob --synth_solv_loop --inv_deg 2


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

Synthesized solvable loop: 
_t22 = d0
_s20 = 3*_u17
d = _t22
while true:
    _t22 = 1/2
    _s20 = (1/2)*_s20
    d = _t22
end

Invariant used: 
(-a**2*_u17 + c**2*_u17, 3*2**(-n)*_u17)


Synthesized solvable loop: 
_t23 = d0
_s21 = _u14 + 3*_u17
d = _t23
while true:
    _t23 = 1/2
    _s21 = (1/2)*_s21
    d = _t23
end

Invariant used: 
(-a*_u14 - a**2*_u17 + c*_u14 + c**2*_u17, 2**(-n)*(_u14 + 3*_u17))


Elapsed time: 0.8217787742614746 s


In [6]:
from inputparser import Parser
from program import normalize_program
from unsolvable_analysis import UnsolvInvSynthesizer

program = Parser().parse_file("loops/markov-triples-random.prob")
program = normalize_program(program)
defective_vars = [v for v in program.original_variables if v in program.defective_variables]
inv_deg = 2
solutions = UnsolvInvSynthesizer.synth_inv(defective_vars, inv_deg, program)
print(solutions)

[(a**2*_u167 - c**2*_u167, -3*2**(-n)*_u167), (a*_u164 + a**2*_u167 - c*_u164 - c**2*_u167, 2**(-n)*(-_u164 - 3*_u167))]


In [9]:
from inputparser import Parser
from program import normalize_program
from unsolvable_analysis import SolvLoopSynthesizer

program = Parser().parse_file("loops/markov-triples-random.prob")
program = normalize_program(program)
defective_vars = [v for v in program.original_variables if v in program.defective_variables]
inv_deg = 2
invariants, solvable_programs = SolvLoopSynthesizer.synth_loop(defective_vars, inv_deg, program)

for invariant, solvable_program in zip(invariants, solvable_programs):
    print("Synthesized solvable loop: ")
    print(solvable_program)
    print()
    print("Invariant used: ")
    print(invariant)
    print()
    print()

Synthesized solvable loop: 
_t229 = d0
_s227 = -3*_u224
d = _t229
while true:
    _t229 = 1/2
    _s227 = (1/2)*_s227
    d = _t229
end

Invariant used: 
(a**2*_u224 - c**2*_u224, -3*2**(-n)*_u224)


Synthesized solvable loop: 
_t230 = d0
_s228 = -_u221 - 3*_u224
d = _t230
while true:
    _t230 = 1/2
    _s228 = (1/2)*_s228
    d = _t230
end

Invariant used: 
(a*_u221 + a**2*_u224 - c*_u221 - c**2*_u224, 2**(-n)*(-_u221 - 3*_u224))


